In [3]:
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import defaultdict

# Define the environment
grid = np.array([
    [ 1,  1,  1,  1,  1],
    [ 1,  1, -1, -1,  1],
    [-1, -1,  1,  1,  1],
    [ 1, -1,  1, -1, -1],
    [ 1,  1,  1, -1, -1],
    [-1, -1,  1,  1, -1],
    [ 2,  1,  1,  1, -1]
])

# Define the possible actions
actions = [0, 1, 2, 3] # up, right, down, left, Terminal
action_move = {
            0: (-1, 0),
            1: (0, 1),
            2: (1, 0),
            3: (0, -1),
        }

# Check if the next state is valid
def is_valid_state(state):
    x, y = state
    if x < 0 or x >= grid.shape[0] or y < 0 or y >= grid.shape[1]:
        return False
    if grid[x, y] == -1:
        return False
    return True
# Get next state

def step(state, action):
    x, y = state
    dx, dy = action_move[action]
    next_state = (x + dx, y + dy)
    if is_valid_state(next_state):
        return next_state
    return state

def calculate_returns(episode, returns, gamma):
    G = 0
    returns = []
    for t in reversed(range(len(episode))):
        state, action, reward = episode[t]
        G = gamma * G + reward
        returns.insert(0, (state, action, G))
        # returns[(state, action)].insert(0, G)
    return returns

def choose_action(epsilon, Q, state):
    if state == (6,0):
        return 4
    r = random.uniform(0,1)
    if r < epsilon:
        action = random.choice(actions)
    else:
        action = max(actions, key=lambda x: Q[(state, x)])
    return action

# Monte Carlo Exploring Starts
def monte_carlo_es(grid, gamma, episodes=1000):

    Q = {}
    returns = {}
    policy = {}
    R_sum = defaultdict(int)
    R_count = defaultdict(int)
    epsilon = 1

    # Initialize state-action pairs
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            if grid[i, j] == 1:
                for action in actions:
                    Q[((i, j), action)] = 0


    for episode in range(episodes):
        # Generate random start state and action
        start_state = (random.randint(0, grid.shape[0] - 1), random.randint(0, grid.shape[1] - 1))
        while not is_valid_state(start_state):
            start_state = (random.randint(0, grid.shape[0] - 1), random.randint(0, grid.shape[1] - 1))
        start_action = choose_action(epsilon, Q, start_state)
        
        # Generate an episode
        episode = []
        state = start_state
        action = start_action
        while grid[state[0], state[1]] != 2:
            next_state = step(state, action)
            reward = 1 if grid[next_state[0], next_state[1]] == 2 else 0
            episode.append((state, action, reward))
            if grid[next_state[0], next_state[1]] == 2:
                episode.append((next_state, 4, 0))
                break
            state = next_state
            action = choose_action(epsilon, Q, state)
            
        # Calculate returns
        returns = calculate_returns(episode, returns, gamma)
        visited_state_action_pairs = set()
        for state, action, ret in returns:
            if state not in visited_state_action_pairs:
                R_sum[(state, action)] += ret
                R_count[(state, action)] += 1     
                visited_state_action_pairs.add((state, action))
    for (state ,action), ret in R_sum.items():
        Q[(state, action)] = ret / R_count[(state, action)]
        if state == (6,0):
            policy[state] = 4
        else:
            policy[state] = max(actions, key=lambda x: Q[(state, x)])
    return policy


emojis = ['⬆️', '➡️', '⬇️', '⬅️', '🏁']
Gammas = [0, .1, .2, .5, .6, .7, .8, .9, 1]
# Print the optimal policy
for gamma in Gammas:
    optimal_policy = monte_carlo_es(grid, gamma)
    print(f"Optimal Policy with gamma = {gamma}:")
    for i in range(grid.shape[0]):
        print('|', end=' ')
        for j in range(grid.shape[1]):
            if grid[i, j] == 1 or grid[i, j] == 2:
                print(emojis[int(optimal_policy[(i, j)])], end=' | ')
            else:
                print('❌', end=' | ')  # Using '❌' for obstacles
        print()



Optimal Policy with gamma = 0:
| ⬆️ | ⬆️ | ⬆️ | ⬆️ | ⬆️ | 
| ⬆️ | ⬆️ | ❌ | ❌ | ⬆️ | 
| ❌ | ❌ | ⬆️ | ⬆️ | ⬆️ | 
| ⬆️ | ❌ | ⬆️ | ❌ | ❌ | 
| ⬆️ | ⬆️ | ⬆️ | ❌ | ❌ | 
| ❌ | ❌ | ⬆️ | ⬆️ | ❌ | 
| 🏁 | ⬅️ | ⬆️ | ⬆️ | ❌ | 
Optimal Policy with gamma = 0.1:
| ➡️ | ➡️ | ➡️ | ➡️ | ⬇️ | 
| ➡️ | ⬆️ | ❌ | ❌ | ⬇️ | 
| ❌ | ❌ | ⬇️ | ⬅️ | ⬅️ | 
| ⬇️ | ❌ | ⬇️ | ❌ | ❌ | 
| ➡️ | ➡️ | ⬇️ | ❌ | ❌ | 
| ❌ | ❌ | ⬇️ | ⬅️ | ❌ | 
| 🏁 | ⬅️ | ⬅️ | ⬅️ | ❌ | 
Optimal Policy with gamma = 0.2:
| ➡️ | ➡️ | ➡️ | ➡️ | ⬇️ | 
| ➡️ | ⬆️ | ❌ | ❌ | ⬇️ | 
| ❌ | ❌ | ⬇️ | ⬅️ | ⬅️ | 
| ⬇️ | ❌ | ⬇️ | ❌ | ❌ | 
| ➡️ | ➡️ | ⬇️ | ❌ | ❌ | 
| ❌ | ❌ | ⬇️ | ⬇️ | ❌ | 
| 🏁 | ⬅️ | ⬅️ | ⬅️ | ❌ | 
Optimal Policy with gamma = 0.5:
| ➡️ | ➡️ | ➡️ | ➡️ | ⬇️ | 
| ⬆️ | ⬆️ | ❌ | ❌ | ⬇️ | 
| ❌ | ❌ | ⬇️ | ⬅️ | ⬅️ | 
| ⬇️ | ❌ | ⬇️ | ❌ | ❌ | 
| ➡️ | ➡️ | ⬇️ | ❌ | ❌ | 
| ❌ | ❌ | ⬇️ | ⬇️ | ❌ | 
| 🏁 | ⬅️ | ⬅️ | ⬅️ | ❌ | 
Optimal Policy with gamma = 0.6:
| ➡️ | ➡️ | ➡️ | ➡️ | ⬇️ | 
| ➡️ | ⬆️ | ❌ | ❌ | ⬇️ | 
| ❌ | ❌ | ⬇️ | ⬅️ | ⬅️ | 
| ⬇️ | ❌ | ⬇️ | ❌ | ❌ | 
| ➡️

In [4]:
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import defaultdict

# Define the environment
grid = np.array([
    [ 1,  1,  1,  1,  1],
    [ 1,  1, -1, -1,  1],
    [-1, -1,  1,  1,  1],
    [ 1, -1,  1, -1, -1],
    [ 1,  1,  1, -1, -1],
    [-1, -1,  1,  1, -1],
    [ 2,  1,  1,  1, -1]
])

# Define the possible actions
actions = [0, 1, 2, 3] # up, right, down, left, Terminal
action_move = {
    0: (-1, 0),
    1: (0, 1),
    2: (1, 0),
    3: (0, -1),
}

# Check if the next state is valid
def is_valid_state(state):
    x, y = state
    if x < 0 or x >= grid.shape[0] or y < 0 or y >= grid.shape[1]:
        return False
    if grid[x, y] == -1:
        return False
    return True

# Get next state
def step(state, action):
    x, y = state
    dx, dy = action_move[action]
    next_state = (x + dx, y + dy)
    if is_valid_state(next_state):
        return next_state
    return state

def calculate_returns(episode, gamma):
    G = 0
    returns = []
    for t in reversed(range(len(episode))):
        state, action, reward = episode[t]
        G = gamma * G + reward
        returns.insert(0, (state, action, G))
    return returns

def choose_action(epsilon, Q, state):
    if state == (6,0):
        return 4
    r = random.uniform(0,1)
    if r < epsilon:
        action = random.choice(actions)
    else:
        action = max(actions, key=lambda x: Q[(state, x)])
    return action

# Monte Carlo Exploring Starts
def monte_carlo_es(grid, gamma, episodes=1000):
    Q = {}
    returns = {}
    policy = {}
    R_sum = defaultdict(int)
    R_count = defaultdict(int)
    epsilon = 1

    # Initialize state-action pairs
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            if grid[i, j] == 1:
                for action in actions:
                    Q[((i, j), action)] = 0

    final_start_state = None  # Track the starting state of the final episode

    for episode in range(episodes):
        # Generate random start state and action
        start_state = (random.randint(0, grid.shape[0] - 1), random.randint(0, grid.shape[1] - 1))
        while not is_valid_state(start_state):
            start_state = (random.randint(0, grid.shape[0] - 1), random.randint(0, grid.shape[1] - 1))
        final_start_state = start_state  # Update final start state for the current episode
        start_action = choose_action(epsilon, Q, start_state)
        
        # Generate an episode
        episode = []
        state = start_state
        action = start_action
        while grid[state[0], state[1]] != 2:
            next_state = step(state, action)
            reward = 1 if grid[next_state[0], next_state[1]] == 2 else 0
            episode.append((state, action, reward))
            if grid[next_state[0], next_state[1]] == 2:
                episode.append((next_state, 4, 0))
                break
            state = next_state
            action = choose_action(epsilon, Q, state)
            
        # Calculate returns
        returns = calculate_returns(episode, gamma)
        visited_state_action_pairs = set()
        for state, action, ret in returns:
            if state not in visited_state_action_pairs:
                R_sum[(state, action)] += ret
                R_count[(state, action)] += 1     
                visited_state_action_pairs.add((state, action))
    
    for (state, action), ret in R_sum.items():
        Q[(state, action)] = ret / R_count[(state, action)]
        if state == (6,0):
            policy[state] = 4
        else:
            policy[state] = max(actions, key=lambda x: Q[(state, x)])
    
    return policy, final_start_state

emojis = ['⬆️', '➡️', '⬇️', '⬅️', '🏁']
start_emoji = '⭐'
Gammas = [0, .1, .2, .5, .6, .7, .8, .9, 1]

# Print the optimal policy
for gamma in Gammas:
    optimal_policy, start_state = monte_carlo_es(grid, gamma)
    print(f"Optimal Policy with gamma = {gamma}:")
    for i in range(grid.shape[0]):
        print('|', end=' ')
        for j in range(grid.shape[1]):
            if (i, j) == start_state:
                print(start_emoji, end=' | ')
            elif grid[i, j] == 1 or grid[i, j] == 2:
                print(emojis[int(optimal_policy[(i, j)])], end=' | ')
            else:
                print('❌', end=' | ')  # Using '❌' for obstacles
        print()


Optimal Policy with gamma = 0:
| ⬆️ | ⬆️ | ⬆️ | ⬆️ | ⬆️ | 
| ⭐ | ⬆️ | ❌ | ❌ | ⬆️ | 
| ❌ | ❌ | ⬆️ | ⬆️ | ⬆️ | 
| ⬆️ | ❌ | ⬆️ | ❌ | ❌ | 
| ⬆️ | ⬆️ | ⬆️ | ❌ | ❌ | 
| ❌ | ❌ | ⬆️ | ⬆️ | ❌ | 
| 🏁 | ⬅️ | ⬆️ | ⬆️ | ❌ | 
Optimal Policy with gamma = 0.1:
| ➡️ | ➡️ | ➡️ | ➡️ | ⬇️ | 
| ➡️ | ⬆️ | ❌ | ❌ | ⬇️ | 
| ❌ | ❌ | ⬇️ | ⬅️ | ⬅️ | 
| ⬇️ | ❌ | ⬇️ | ❌ | ❌ | 
| ➡️ | ➡️ | ⬇️ | ❌ | ❌ | 
| ❌ | ❌ | ⭐ | ⬇️ | ❌ | 
| 🏁 | ⬅️ | ⬅️ | ⬅️ | ❌ | 
Optimal Policy with gamma = 0.2:
| ⬇️ | ➡️ | ➡️ | ➡️ | ⬇️ | 
| ➡️ | ⬆️ | ❌ | ❌ | ⬇️ | 
| ❌ | ❌ | ⬇️ | ⬅️ | ⬅️ | 
| ⬇️ | ❌ | ⬇️ | ❌ | ❌ | 
| ➡️ | ➡️ | ⭐ | ❌ | ❌ | 
| ❌ | ❌ | ⬇️ | ⬇️ | ❌ | 
| 🏁 | ⬅️ | ⬅️ | ⬅️ | ❌ | 
Optimal Policy with gamma = 0.5:
| ➡️ | ➡️ | ➡️ | ➡️ | ⬇️ | 
| ➡️ | ⬆️ | ❌ | ❌ | ⬇️ | 
| ❌ | ❌ | ⬇️ | ⬅️ | ⬅️ | 
| ⬇️ | ❌ | ⭐ | ❌ | ❌ | 
| ➡️ | ➡️ | ⬇️ | ❌ | ❌ | 
| ❌ | ❌ | ⬇️ | ⬇️ | ❌ | 
| 🏁 | ⬅️ | ⬅️ | ⬅️ | ❌ | 
Optimal Policy with gamma = 0.6:
| ⭐ | ➡️ | ➡️ | ➡️ | ⬇️ | 
| ➡️ | ⬆️ | ❌ | ❌ | ⬇️ | 
| ❌ | ❌ | ⬇️ | ⬅️ | ⬅️ | 
| ⬇️ | ❌ | ⬇️ | ❌ | ❌ | 
| ➡️ | ➡️